In [13]:
import requests
from bs4 import BeautifulSoup
import time
import os
import re

def safe_filename(name):
    # 파일에 사용할 수 없는 문자 제거 또는 치환
    return re.sub(r'[\\/:"*?<>|\'"]+', '_', name).strip()

base_url = "https://hyu.wiki"
from_value = ""

while True:
    full_url = f"{base_url}/index.php?title=특수:모든문서&from={from_value}&hideredirects=1"
    response = requests.get(full_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    titles = soup.select('.mw-allpages-chunk > li > a')
    for t in titles:
        sub_url = base_url + t['href']
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        
        head = sub_soup.select_one('.firstHeading').text
        temp = safe_filename(head)
        
        save_dir = "/Users/shinbanseok/Desktop/HYU_ChatBot/data"
        os.makedirs(save_dir, exist_ok=True)
        file_name = f"{temp}.txt"
        file_path = os.path.join(save_dir, file_name)

        # 파일이 존재하지 않을 경우에만 실행
        if not os.path.exists(file_path):
            container = sub_soup.select('.mw-parser-output')
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(f"{head}\n")  # 문서 제목 한 번만 기록
                for tag in container:
                    text = tag.text
                    if text:
                        f.write(f"{text}")
            # time.sleep(0.1)
        else:
            print(f"✅ 이미 존재하는 파일: {file_name}, 건너뜀")

    next_link = soup.select_one('.mw-allpages-nav > a:nth-of-type(2)')
    if next_link:
        href = next_link['href']
        from_value = href.split("from=")[1].split("&")[0]
    else:
        break

✅ 이미 존재하는 파일: 21c개발경영정책과정.txt, 건너뜀
✅ 이미 존재하는 파일: Angelus.txt, 건너뜀
